In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

많은 용어들을 정확히 정의하고 이해해야 타 연구자들과 커뮤니케이션에서 문제가 없다.

## 8. Sequential Models - Transformer


### problem

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/105732.png?raw=ture" width="500"> 

위와같은 Sequential 데이터의 문제점을 해결하기 위해 Attention 및 Transformer가 등장하였다.

### 8-1. Transformer

Attention 구조를 사 용한 모델

* RNN을 시간순으로 쭉 풀게되면 입력이 굉장히 많은 Fully Connected Layer로 만들 수 있다.

#### 8-1-1. Sequence to Sequence

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/111249.png?raw=ture" width="500"> 

기계번역에서 문장을 문장으로(StoS) 바로 번역해주는 방식

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/112407.png?raw=ture" width="500"> 

Transformer 구조에서 핵심은 **self-Attention** 구조(feed-foward는 MLP구조와 동일)

#### Self-Attetion

N개의 단어를 만들려고 할때, ${x_i}$을 ${z_i}$로 바꿀때 나머지 ${x_{n-1}}$ 개의 단어를 같이 고려하는 점이 있다.**(dependencies)**

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/113037.png?raw=ture" width="500"> 

* 예시 : ${x_1}$ 을 입력으로 받아 ${z_1}$ 을 생성할때 ${x_2,x_3}$ 와 같이 고려한다.

다만 feed-forward NN는 Self-Attention과 달리 받은 입력을 그대로 변환해 출력을 해준다.**(inpendent)** 

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/114129.png?raw=ture" width="500"> 

생각하자면, 위와같은 문장이 들어올때 **it**은 문장내에 전체적인 분위기에 맞춰 해석되어야 하는데, Self-Attetion이 그 역할을 해준다.

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/self-attention-output.png?raw=ture" width="500"> 

- Data $X \in \mathbb{R}^{n \times d}$ where $n$ is the number data and $d$ is the data dimension

ex) d는 word embedding vector의 차원
- Query and Key $Q, K \in \mathbb{R}^{n \times d_K}$ 
- Value $V \in \mathbb{R}^{n \times d_V} $

$\text{Attention}(Q,K,V) = \text{softmax} \left( \frac{QK^T}{\sqrt{d_K}} \right)V \in \mathbb{R}^{n \times d_V} $

Queries, Keys, Values 각 3개의 벡터는 3개의 NN라고 생각하면 된다.

* Score : ${q_1 \bullet k_1}$ 쿼리와 키값의 내적으로 구함

* nomalize of Score : ${ \sqrt d_k}$ (Key vector의 dim) 여기서는 ${d_k}$ = 64 , Score의 범위를 어느정도 한정짓기 위함

* Softmax : Thinking은 자기자신과의 interaction이 0.88 Machine과의 interaction은 0.12

* Softmax * Value : 계산한 Softmax 값과 value vectors을 weighted sum(가중합)을 취한다. 

* Sum : 모든 encoding을 거치면 하나의 단어에 대한 ${z_1}$이 나오게 된다.

#### 참고사항
1. Qurey vectors 와 Key vectors는 내적을 해야하므로 차원이 같아야 한다.

2. 그러나 value vectors는 weighted sum 만 하면 되므로 차원이 달라도 된다.

3. 위 자료를 기준으로 ${z_1}$ (sum)은 Value vecotrs의 차원과 동일하다.

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/self-attention-matrix-calculation.png?raw=ture" width="500"> 

* ${X}$: 2x4 2개의 단어와 4개의 임베딩 벡터
* ${W^Q, W^K, W^V}$ : Q,K,V를 구하는 MLP

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/self-attention-matrix-calculation-2.png?raw=ture" width="500">


#### 왜 이게 잘될까?
MLP, CNN에 Image를 넣는다고 생각하면 입력이 고정되어 있으면 출력도 고정된다.

그러나 Transformer는 입력 하나가 고정되어도 그 외의 다른 입력들에 의해서 출력이 고정되지 않을 수 있다. 좀 더 flexible하고 많은 걸 표현할 수 있는 방법

* RNN과의 비교 : 천개의 단어가 들어가면 트랜스포머는 1000x1000 matrix를 한번에 계산 할 수 있지만, RNN은 천번 반복해서 결과를 얻어낸다. Length가 길어짐에 따라서 트랜스포머는 한번에 처리해야하는게 많아져 상대적으로 RNN에 비해 메모리를 많이 먹게 된다.

#### MHA(Mulit-Head-Attetion)

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/transformer_attention_heads_qkv.png?raw=ture" width="500">

MHA은 하나의 단어에 대해서 여러개의 Attention을 붙여준다.

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/MHA.png?raw=ture" width="500">

여러개의 Attention이 나오다보니 입력 값과 차원이 달라지게 된다

* 여기선 8번의 결과값이 붙게 된다. 이를 행렬곱을 통해서 다시 원래의 차원으로 돌려줘야 한다.

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/transformer_multi-headed_self-attention-recap.png?raw=ture" width="500">


#### Positional encoding

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/positional.png?raw=ture" width="500">

Positional encoding이 필요한 이유는 문장의 순서정보가 Attetion 연산만으로는 없기때문에 문장의 순서 정보를 대신하는 역할을 한다.

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/transformer_positional_encoding_example.png?raw=ture" width="500">

* Positional encoding 예시

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/512dim.png?raw=ture" width="500">

* 512 dim의 입력의 예시

#### Transformer-encoder

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/layernorm_etc.png?raw=ture" width="500">

위에서 계속 학습한 Attention 이 지나가면 Layernorm등을 지나 Feed-Forward(MLP)등의 연산이 계속된다.


#### Transformer-decoder

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/transformer_resideual_layer_norm_3.png?raw=ture" width="500">

encoder를 통해서 입력된 단어의 Key & Value vectors를 decoder로 전달하한다.

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/encoderdecoder.png?raw=ture" width="500">

그리고 decoder에서는 입력으로 받는 Key & Value vectors를 auto-regressive(한 단어씩)하게 출력한다.

#### masking

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/masking.png?raw=ture" width="500">

masking은 학습단계에서 미래의 정보들을 미리알고 학습하는것은 의미가 없기 때문에 Decoder의 Self-Attetion 부분에서 뒤에 있는 정보를 함께 활용하지 않고 학습을 하게 하는 것이다. 

* 추론시에도 마스킹한 상태로 진행한다.

#### Encoder-Decoder Attention

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/encoderdecoder_2.png?raw=ture" width="500">

Encoder와 Decoder사이에 interaction과 지금 Decoder에 들어간 단어들만 가지고 Query를 만들고 Key,Value는 Encoder에서 들어온 것을 활용한다.

#### Transformer output

<img src="https://github.com/dongseoklee1541/boostcamp_AI_Tech_3/blob/main/images/transformer_decoder_output_softmax.png?raw=ture" width="500">


이미지 출처 : http://jalammar.github.io/illustrated-transformer/